Imports.

In [ ]:
import sys
import warnings
from pathlib import Path
import numpy as np
import pandas as pd

In [ ]:
D_ECHEM_LABELS = dict(time="TestTime", 
                      voltage="Voltage [V]",
                      current="Current [A]",
                      step="Step"
                      )

Ignore UserWarnings in case `sys.exit()` is encountered.

In [ ]:
warnings.filterwarnings("ignore", category=UserWarning)

Checking whether a folder called `data` exists.

In [ ]:
data_path = Path.cwd() / "data"
if not data_path.exists():
    data_path.mkdir()
    s = f"\n{80*'-'}\nA folder called '{data_path.name}' has been created."
    s += f"\nPlease put your data files there and rerun the code.\n{80*'-'}"
    sys.exit(s)

Checking that data files are present in the `data` folder.

In [ ]:
data_files = list(data_path.glob("*.*"))
if len(data_files) == 0:
    s = f"\n{80*'-'}\nNo files were found in the '{data_path.name}' folder.\n"
    s += f"Please put your data files there and rerun the code.\n{80*'-'}"
    sys.exit(s)

Function to get header and index for header line in the data file.

In [ ]:
def get_header_idx(data_file):
    with data_file.open(mode="r") as f:
        lines = f.readlines()
    for i, line in enumerate(lines):
        if "Rec" in line:
            break

    return i, lines[i].strip().split("\t")

Function to write the extracted data to a `.txt` file.

In [ ]:
def write(df, d, filename, output_path):
    time = df[d["time"]].to_numpy()
    time = (time - time[0]) / 60
    step = df[d["step"]].to_numpy()
    voltage, current = df[d["voltage"]].to_numpy(), df[d["current"]].to_numpy()
    sign_current = []
    for e in step:
        if e == 5:
            sign_current.append(1)
        else:
            sign_current.append(-1)
    current = current * np.array(sign_current)
    import matplotlib.pyplot as plt
    # plt.plot(time, voltage),
    plt.plot(time, current)
    header = "t [h]\tV [V]\ti [A]"
    np.savetxt(output_path / filename,
               np.column_stack((time, voltage, current)),
               delimiter="\t",
               header=header,
               fmt="%.8f",
               encoding="utf8",
               )
    
    return None

Making sure that the output path exists.

In [ ]:
txt_path = Path.cwd() / "txt"
if not txt_path.exists():
    txt_path.mkdir()

Extracting data and writing to `.txt` file.

In [ ]:
print(f"{80*'-'}\nWorking with files...")
for f in data_files:
      print(f"\t{f.name}")
      header_idx, header = get_header_idx(f)
      df = pd.read_csv(f,
                       skiprows=header_idx+1,
                       delimiter="\t",
                       names=header,
                       usecols=[i for i in range(len(header))],
                       )
      write(df, D_ECHEM_LABELS, f.name, txt_path)
print(f"Done working with files. Please see the '{txt_path.name}' folder."
      f"\n{80*'-'}")